In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

### Dataset

In [2]:
data = pd.read_csv('lithium_sentiment.csv', index_col=0)

In [3]:
data['Price'] = data['Price'] / 1000
data

,Date,Price,VIX,news
0,2017-05-10,136.0,10.21,0.000000
1,2017-05-11,136.0,10.60,-0.108700
2,2017-05-12,136.0,10.40,-0.108700
3,2017-05-15,136.0,10.42,-0.557400
4,2017-05-16,136.0,10.65,0.157825
...,...,...,...,...
1530,2023-09-21,173.5,17.54,-0.051467
1531,2023-09-22,172.5,17.20,0.138950
1532,2023-09-25,169.5,16.90,-0.029167
1533,2023-09-26,169.5,18.94,-0.208300


In [4]:
def lagged_seq(seq, lag, h):
    X = [seq[i:i+lag] for i in range(len(seq) - lag - h + 1)]
    y = [seq[i+lag:i+lag+h] for i in range(len(seq) - lag - h + 1)]
    return X, y

In [5]:
def lagged_seq_sent(seq, lag, h, sent, sent_lag):
    X = [seq[i:i+lag] + sent[i+lag-sent_lag: i+lag] for i in range(len(seq) - lag - h + 1)]    
    y = [seq[i+lag:i+lag+h] for i in range(len(seq) - lag - h + 1)]
    return X, y

In [6]:
class TensorData():
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X) 
        self.y = torch.FloatTensor(y)
        self.len = self.y.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index] 

    def __len__(self):
        return self.len

In [7]:
def split(X, y, train_size=0.7, val_size=0.1, random_state=0):
    tl = int(len(X) * train_size)
    vl = int(len(X) * val_size)

    X_train, y_train = X[:tl], y[:tl]
    X_val, y_val = X[tl:tl + vl], y[tl:tl + vl]   
    X_test, y_test = X[tl + vl:], y[tl + vl:]
    
    X_max = X_train.max()
    X_min = X_train.min()
    y_max = y_train.max()
    y_min = y_train.min()
    
    
    X_train = (X_train - X_min)/(X_max - X_min)
    X_train = torch.FloatTensor(X_train)
    y_train = (y_train - y_min)/(y_max - y_min)
    y_train = torch.FloatTensor(y_train)
 
    X_val = (X_val - X_min)/(X_max - X_min)
    X_val = torch.FloatTensor(X_val)
    y_val = (y_val - y_min)/(y_max - y_min)
    y_val = torch.FloatTensor(y_val)
    
    X_test = (X_test - X_min)/(X_max - X_min)
    X_test = torch.FloatTensor(X_test)
    y_test = (y_test - y_min)/(y_max - y_min)
    y_test = torch.FloatTensor(y_test)
    
    y_max = torch.FloatTensor(y_max)
    y_min = torch.FloatTensor(y_min)
    
    return X_train, y_train, X_val, y_val, X_test, y_test, y_max, y_min

In [8]:
def tensor_data_split(lag, data, h, s=False, sent_lag=0, sent_type=None):
    daily = list(data['Price'])

    if s:
        sent = list(data[sent_type])
        X, y = lagged_seq_sent(daily, lag, h, sent, sent_lag)
    else:
        X, y = lagged_seq(daily, lag, h)
        
    ts = TensorData(X, y)
    return split(ts.X, ts.y)

In [9]:
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out

In [10]:
def train_(model, X_train, y_train, X_val, y_val, optimizer, num_epochs, scheduler=False, batch_size):

    model = model.to(device)
    criterion = nn.MSELoss()

    X_train = X_train.to(device)  
    y_train = y_train.to(device)
    X_val = X_val.to(device)     
    y_val = y_val.to(device)

    best_val_mse = float('inf')  # Initialize the best validation MSE
    consecutive_increases = 0
    
    if not scheduler:
        for epoch in range(num_epochs):
            model.train()
            for i in range(0,len(X_train), batch_size):
                X_batch = X_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if (epoch+1) % 1000 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()} ') 

                model.eval()
                with torch.no_grad():
                    val_outputs = model(X_val)

                val_mse = criterion(val_outputs, y_val).item()
                # print(f"Validation MSE: {val_mse}\n")
                
                if epoch > 5000:

                    if val_mse < best_val_mse:
                        best_val_mse = val_mse
                        consecutive_increases = 0  # Reset consecutive increases
                    else:
                        consecutive_increases += 1

                    if consecutive_increases >= 3:
                        # print(f"Stopping early due to increasing validation MSE for 3 consecutive epochs.")
                        break
                    
    else:
        for epoch in range(num_epochs):
            if epoch % 500 != 0:
                model.train()
                for i in range(0,len(X_train), batch_size):
                    X_batch = X_train[i:i+batch_size]
                    y_batch = y_train[i:i+batch_size]
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                if (epoch+1) % 1000 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()} ') 

                    model.eval()
                    with torch.no_grad():
                        val_outputs = model(X_val)

                    val_mse = criterion(val_outputs, y_val).item()
                    # print(f"Validation MSE: {val_mse}\n")

                    if epoch > 5000:

                        if val_mse < best_val_mse:
                            best_val_mse = val_mse
                            consecutive_increases = 0  # Reset consecutive increases
                        else:
                            consecutive_increases += 1

                        if consecutive_increases >= 3:
                            # print(f"Stopping early due to increasing validation MSE for 3 consecutive epochs.")
                            break
                            
            if epoch % 500 == 0:
                model.train()
                for i in range(0,len(X_train), batch_size):
                    X_batch = X_train[i:i+batch_size]
                    y_batch = y_train[i:i+batch_size]
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                if (epoch+1) % 1000 == 0:
                    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()} ') 

                    model.eval()
                    with torch.no_grad():
                        val_outputs = model(X_val)

                    val_mse = criterion(val_outputs, y_val).item()
                    # print(f"Validation MSE: {val_mse}\n")

                    if epoch > 5000:

                        if val_mse < best_val_mse:
                            best_val_mse = val_mse
                            consecutive_increases = 0  # Reset consecutive increases
                        else:
                            consecutive_increases += 1

                        if consecutive_increases >= 3:
                            # print(f"Stopping early due to increasing validation MSE for 3 consecutive epochs.")
                            break

    print(f"Stopping at epoch {epoch-1} with Validation MSE: {val_mse}")
    return best_val_mse

In [11]:
def eval(model, X_test, y_test):

    model = model.to(device)
    X_test = X_test.to(device)  
    y_test = y_test.to(device)

    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        
    loss = nn.MSELoss()(y_test, y_pred)
    
    print(f"== Test MSE: {loss.item()} ==")
    # print('')
    
    y_pred = y_pred.cpu()
    
    return y_pred, loss.item()

In [12]:
def real_pred_mse(pred, true, y_max, y_min):
    real_true = true*(y_max-y_min) + y_min
    real_pred = pred*(y_max-y_min) + y_min
    loss = nn.MSELoss()(real_true, real_pred)
    print(f"== Original Test MSE: {loss.item()} ==")
    return real_pred, loss.item()

In [13]:
def plot_(data, y_pred):
    fig, ax = plt.subplots(figsize=(10, 3))
    
    ax.plot(data['Price'], label='True')
    
    x_pred = np.arange(len(data) - len(y_pred), len(data))
    ax.plot(x_pred, y_pred.cpu().detach().numpy(), label='Predict', linestyle='--')
    
    ax.legend()
    ax.grid()
    plt.show()

In [14]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

## Naive MSE

In [15]:
test = data.iloc[1228:]

In [16]:
prev_y_1 = data.iloc[1227:-1]
naive_y_1 = [[i]*1 for i in prev_y_1['Price'].values]
true_y = torch.FloatTensor([test['Price'][i:i+1].values for i in range(test['Price'].shape[0])])
print(f'Naive MSE (h=1): {nn.MSELoss()(torch.FloatTensor(naive_y_1), true_y)}')

Naive MSE (h=1): 20.869707107543945


/tmp/ipykernel_37186/2045318868.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541990/work/torch/csrc/utils/tensor_new.cpp:230.)
  true_y = torch.FloatTensor([test['Price'][i:i+1].values for i in range(test['Price'].shape[0])])


In [17]:
prev_y_5 = data.iloc[1227:-5]
naive_y_5 = [[i]*5 for i in prev_y_5['Price'].values]
true_y_5 = torch.FloatTensor([test['Price'][i:i+5].values for i in range(test['Price'].shape[0]-4)])
print(f'Naive MSE (h=5): {nn.MSELoss()(torch.FloatTensor(naive_y_5), true_y_5)}')

Naive MSE (h=5): 168.0389404296875


In [18]:
prev_y_10 = data.iloc[1227:-10]
naive_y_10 = [[i]*10 for i in prev_y_10['Price'].values]
true_y_10 = torch.FloatTensor([test['Price'][i:i+10].values for i in range(test['Price'].shape[0]-9)])
print(f'Naive MSE (h=10): {nn.MSELoss()(torch.FloatTensor(naive_y_10), true_y_10)}')

Naive MSE (h=10): 517.5758666992188


In [19]:
prev_y_22 = data.iloc[1227:-22]
naive_y_22 = [[i]*22 for i in prev_y_22['Price'].values]
true_y_22 = torch.FloatTensor([test['Price'][i:i+22].values for i in range(test['Price'].shape[0]-21)])
print(f'Naive MSE (h=22): {nn.MSELoss()(torch.FloatTensor(naive_y_22), true_y_22)}')

Naive MSE (h=22): 1989.8699951171875


## Pure ANN

In [20]:
d = tensor_data_split(22, data, 1)

state_dict = torch.load('./models/sc/_h1_ANN_')
    
lm = ANN(len(d[0][0]), 10, 1)
lm.load_state_dict(state_dict)
# e = eval(lm, d[-4], d[-3])
# e = e[0].cpu()
# r = real_pred_mse(e, d[-3], d[-2], d[-1])

<All keys matched successfully>

### Setting

In [21]:
no_sent = {}
vix = {}
bloom = {}

for h in [1, 5, 10, 22]:
    no_sent[h] = tensor_data_split(22, data, h)
    vix[h] = tensor_data_split(22, data, h, True, 1, 'VIX')
    bloom[h] = tensor_data_split(22, data, h, True, 1, 'news')

In [22]:
def iterate(sent_type, num_epochs, h, lr,weight_decay = 0, scheduler=False):
    print(len(sent_type[h][0][0]))
        
    if h==10:
        hidden = 20
    elif h==22:
        hidden = 40
    else:
        hidden = 10
    
    model = ANN(sent_type[h][0][0].shape[0], hidden, h)
    
    if scheduler:
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500*x for x in range(1, 2000)], gamma=0.1)
        train_(model, sent_type[h][0],sent_type[h][1],sent_type[h][2], sent_type[h][3],
              optimizer, num_epochs, scheduler)
        pred, mse = eval(model, sent_type[h][-4], sent_type[h][-3])
        loss = real_pred_mse(pred, sent_type[h][-3], sent_type[h][-2], sent_type[h][-1])[-1]
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        train_(model, sent_type[h][0],sent_type[h][1],sent_type[h][2], sent_type[h][3],
              optimizer, num_epochs)
        pred, mse = eval(model, sent_type[h][-4], sent_type[h][-3])
        loss = real_pred_mse(pred, sent_type[h][-3], sent_type[h][-2], sent_type[h][-1])[-1]
        
    return model, loss

In [23]:
[1,2,3] < [2,1,4]

True

In [24]:
tables_no = [[],[],[],[]]
tables_vix = [[],[],[],[]]
tables_bloom = [[],[],[],[]]
tables_no_sch = [[],[],[],[]]
tables_vix_sch = [[],[],[],[]]
tables_bloom_sch = [[],[],[],[]]
tables = [[],[],[],[]]
tables_sch = [[],[],[],[]]
#d_name = ['ANN', 'VIX', 'Bloomberg']


num_epochs=1000000


#while True:
naive_mse = [21.15, 168.04, 510.74, 1903.75]
temp_all = []
temp_all_sch = []

for lr in np.arange(0.001, 0.1, 0.001):
    lr = round(lr,3)
    for wd in [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]:
        temps = []
        temps_sch = []
        
        for idx, h in enumerate([1,5,10,22]):
            print('lr: ', lr, 'wd: ', wd, 'h: ',h)

            print('ANN')
            temp_m_no, temp_mse_no = iterate(no_sent, num_epochs, h, lr, wd)
            temp_m_sch, temp_mse_sch = iterate(no_sent, num_epochs, h, lr, wd)


            print('VIX')
            temp_m_no_vix, temp_mse_no_vix = iterate(vix, num_epochs, h, lr, wd)
            temp_m_sch_vix, temp_mse_sch_vix = iterate(vix, num_epochs, h, lr, wd)
 

            print('Bloomberg')
            temp_m_no_bloom, temp_mse_no_bloom = iterate(bloom, num_epochs, h, lr, wd)
            temp_m_sch_bloom, temp_mse_sch_bloom = iterate(bloom, num_epochs, h, lr, wd)

            print(f'naive mse: {naive_mse[0]}')


            if (temp_mse_no < naive_mse[0]) & (temp_mse_no_vix < naive_mse[0]) & (temp_mse_no_bloom < naive_mse[0]):
                torch.save(temp_m_no.state_dict(), f'./models/sy/all/1107_h{h}_ANN_wd{wd}_lr{lr}')
                torch.save(temp_m_no_vix.state_dict(), f'./models/sy/all/1107_h{h}_VIX_wd{wd}_lr{lr}')
                torch.save(temp_m_no_bloom.state_dict(), f'./models/sy/all/1107_h{h}_Bloomberg_wd{wd}_lr{lr}')
                temps.append([temp_mse_no, temp_mse_no_vix, temp_mse_no_bloom, lr, wd, h])
                

            if (temp_mse_sch < naive_mse[0]) & (temp_mse_sch_vix < naive_mse[0]) & (temp_mse_sch_bloom < naive_mse[0]):
                torch.save(temp_m_sch.state_dict(), f'./models/sy/all/1107_h{h}_ANN_wd{wd}_lr{lr}_sch')
                torch.save(temp_m_sch_vix.state_dict(), f'./models/sy/all/1107_h{h}_VIX_wd{wd}_lr{lr}_sch')
                torch.save(temp_m_sch_bloom.state_dict(), f'./models/sy/all/1107_h{h}_Bloomberg_wd{wd}_lr{lr}_sch')
                temps_sch.append([temp_mse_sch, temp_mse_sch_vix, temp_mse_sch_bloom, lr, wd, h])
                
            
            else:
                print('retry')
                
        if len(temps) == 4:
            temp_all.append(temps)
        if len(temps_sch) == 4:
            temp_all_sch.append(temps_sch)
            
print(temp_all)
print(temp_all_sch)


lr:  0.001 wd:  0.001 h:  1
ANN
22
Stopping at epoch 7818 with Validation MSE: 0.0033169889356940985
== Test MSE: 0.004379496909677982 ==
== Original Test MSE: 106.57945251464844 ==
22
Stopping at epoch 2848 with Validation MSE: 0.006729464512318373
== Test MSE: 0.004983818158507347 ==
== Original Test MSE: 121.28621673583984 ==
VIX
23
Stopping at epoch 7948 with Validation MSE: 0.0036335489712655544
== Test MSE: 0.004782871343195438 ==
== Original Test MSE: 116.39593505859375 ==
23
Stopping at epoch 11038 with Validation MSE: 0.003725995309650898
== Test MSE: 0.00474134786054492 ==
== Original Test MSE: 115.38546752929688 ==
Bloomberg
23
Stopping at epoch 8538 with Validation MSE: 0.0038347835652530193
== Test MSE: 0.0048468634486198425 ==
== Original Test MSE: 117.9532699584961 ==
23
Stopping at epoch 10728 with Validation MSE: 0.0038097051437944174
== Test MSE: 0.004856155253946781 ==
== Original Test MSE: 118.17940521240234 ==
naive mse: 21.15
retry
lr:  0.001 wd:  0.001 h:  5
ANN

Stopping at epoch 4118 with Validation MSE: 0.02754853665828705
== Test MSE: 0.024913297966122627 ==
== Original Test MSE: 606.2900390625 ==
23
Stopping at epoch 10228 with Validation MSE: 0.0011154341045767069
== Test MSE: 0.0018298209179192781 ==
== Original Test MSE: 44.53053283691406 ==
naive mse: 21.15
retry
lr:  0.001 wd:  1e-05 h:  5
ANN
22
Stopping at epoch 3028 with Validation MSE: 0.011515744030475616
== Test MSE: 0.007326674181967974 ==
== Original Test MSE: 178.30194091796875 ==
22
Stopping at epoch 2718 with Validation MSE: 0.019642887637019157
== Test MSE: 0.015799637883901596 ==
== Original Test MSE: 384.5 ==
VIX
23
Stopping at epoch 3108 with Validation MSE: 0.013031594455242157
== Test MSE: 0.016712747514247894 ==
== Original Test MSE: 406.7214050292969 ==
23
Stopping at epoch 6078 with Validation MSE: 0.04000331461429596
== Test MSE: 0.02162685990333557 ==
== Original Test MSE: 526.311279296875 ==
Bloomberg
23
Stopping at epoch 2938 with Validation MSE: 0.01855123043

Stopping at epoch 2918 with Validation MSE: 0.044679906219244
== Test MSE: 0.03827671334147453 ==
== Original Test MSE: 931.5020751953125 ==
23
Stopping at epoch 3498 with Validation MSE: 0.04363902658224106
== Test MSE: 0.04054229333996773 ==
== Original Test MSE: 986.6372680664062 ==
Bloomberg
23
Stopping at epoch 40438 with Validation MSE: 0.002598433056846261
== Test MSE: 0.003406466916203499 ==
== Original Test MSE: 82.89977264404297 ==
23
Stopping at epoch 8128 with Validation MSE: 0.0050193145871162415
== Test MSE: 0.004877082537859678 ==
== Original Test MSE: 118.68866729736328 ==
naive mse: 21.15
retry
lr:  0.001 wd:  1e-07 h:  10
ANN
22
Stopping at epoch 2028 with Validation MSE: 0.046184077858924866
== Test MSE: 0.05475151166319847 ==
== Original Test MSE: 1332.4329833984375 ==
22
Stopping at epoch 2038 with Validation MSE: 0.025263814255595207
== Test MSE: 0.029211455956101418 ==
== Original Test MSE: 710.8899536132812 ==
VIX
23
Stopping at epoch 2098 with Validation MSE: 

Stopping at epoch 1698 with Validation MSE: 0.0429258719086647
== Test MSE: 0.04853985458612442 ==
== Original Test MSE: 1181.2659912109375 ==
22
Stopping at epoch 1638 with Validation MSE: 0.07712887227535248
== Test MSE: 0.07888645678758621 ==
== Original Test MSE: 1919.781005859375 ==
VIX
23
Stopping at epoch 1748 with Validation MSE: 0.05668811872601509
== Test MSE: 0.056899238377809525 ==
== Original Test MSE: 1384.699951171875 ==
23
Stopping at epoch 1978 with Validation MSE: 0.0386553630232811
== Test MSE: 0.04206029325723648 ==
== Original Test MSE: 1023.5792846679688 ==
Bloomberg
23
Stopping at epoch 2058 with Validation MSE: 0.03200017660856247
== Test MSE: 0.03718375787138939 ==
== Original Test MSE: 904.904052734375 ==
23
Stopping at epoch 1988 with Validation MSE: 0.04089239984750748
== Test MSE: 0.043255433440208435 ==
== Original Test MSE: 1052.6641845703125 ==
naive mse: 21.15
retry
lr:  0.002 wd:  0.0001 h:  22
ANN
22
Stopping at epoch 1888 with Validation MSE: 0.1322

Stopping at epoch 1718 with Validation MSE: 0.0757119432091713
== Test MSE: 0.0858355164527893 ==
== Original Test MSE: 2088.893310546875 ==
23
Stopping at epoch 6358 with Validation MSE: 0.009758638218045235
== Test MSE: 0.02796064503490925 ==
== Original Test MSE: 680.4503173828125 ==
naive mse: 21.15
retry
lr:  0.002 wd:  1e-06 h:  22
ANN
22
Stopping at epoch 1728 with Validation MSE: 0.656718373298645
== Test MSE: 0.693575918674469 ==
== Original Test MSE: 17314.4296875 ==
22
Stopping at epoch 1968 with Validation MSE: 0.32500210404396057
== Test MSE: 0.31146371364593506 ==
== Original Test MSE: 7775.38037109375 ==
VIX
23
Stopping at epoch 1908 with Validation MSE: 0.19421158730983734
== Test MSE: 0.16744346916675568 ==
== Original Test MSE: 4180.05908203125 ==
23
Stopping at epoch 2258 with Validation MSE: 0.12398256361484528
== Test MSE: 0.14594964683055878 ==
== Original Test MSE: 3643.487060546875 ==
Bloomberg
23
Stopping at epoch 1968 with Validation MSE: 0.07064174860715866


...

Stopping at epoch 8418 with Validation MSE: 3.939403772354126
== Test MSE: 4.46445894241333 ==
== Original Test MSE: 111450.7578125 ==
22
Stopping at epoch 4068 with Validation MSE: 0.04109480232000351
== Test MSE: 0.06017671152949333 ==
== Original Test MSE: 1502.2510986328125 ==
VIX
23
Stopping at epoch 1608 with Validation MSE: 3.9394025802612305
== Test MSE: 4.4633941650390625 ==
== Original Test MSE: 111424.171875 ==
23
Stopping at epoch 10018 with Validation MSE: 0.04026757925748825
== Test MSE: 0.06472118943929672 ==
== Original Test MSE: 1615.69970703125 ==
Bloomberg
23
Stopping at epoch 4818 with Validation MSE: 0.20951412618160248
== Test MSE: 0.14111609756946564 ==
== Original Test MSE: 3522.822021484375 ==
23
Stopping at epoch 8568 with Validation MSE: 3.9394023418426514
== Test MSE: 4.464457035064697 ==
== Original Test MSE: 111450.7109375 ==
naive mse: 21.15
retry
lr:  0.098 wd:  0.001 h:  1
ANN
22
Stopping at epoch 2198 with Validation MSE: 2.4823317527770996
== Test MS

Stopping at epoch 5488 with Validation MSE: 0.0011328889522701502
== Test MSE: 0.0015954982955008745 ==
== Original Test MSE: 38.82803726196289 ==
22
Stopping at epoch 8328 with Validation MSE: 4.215227127075195
== Test MSE: 4.376634120941162 ==
== Original Test MSE: 106509.765625 ==
VIX
23
Stopping at epoch 7428 with Validation MSE: 0.0012181188212707639
== Test MSE: 0.0018299557268619537 ==
== Original Test MSE: 44.53379821777344 ==
23
Stopping at epoch 7128 with Validation MSE: 0.003993071615695953
== Test MSE: 0.004189963452517986 ==
== Original Test MSE: 101.96699523925781 ==
Bloomberg
23
Stopping at epoch 5208 with Validation MSE: 0.004824885632842779
== Test MSE: 0.004485258832573891 ==
== Original Test MSE: 109.1532974243164 ==
23
Stopping at epoch 3948 with Validation MSE: 4.214656352996826
== Test MSE: 4.376071929931641 ==
== Original Test MSE: 106496.0703125 ==
naive mse: 21.15
retry
lr:  0.098 wd:  1e-05 h:  5
ANN
22
Stopping at epoch 4708 with Validation MSE: 0.0040115839

Stopping at epoch 4398 with Validation MSE: 0.005903444718569517
== Test MSE: 0.008754728361964226 ==
== Original Test MSE: 213.05503845214844 ==
22
Stopping at epoch 5418 with Validation MSE: 0.005137520842254162
== Test MSE: 0.009207645431160927 ==
== Original Test MSE: 224.07720947265625 ==
VIX
23
Stopping at epoch 15788 with Validation MSE: 4.16949987411499
== Test MSE: 4.41987419128418 ==
== Original Test MSE: 107562.0546875 ==
23
Stopping at epoch 4758 with Validation MSE: 0.032344598323106766
== Test MSE: 0.04058891907334328 ==
== Original Test MSE: 987.7717895507812 ==
Bloomberg
23
Stopping at epoch 14008 with Validation MSE: 4.16917610168457
== Test MSE: 4.419544696807861 ==
== Original Test MSE: 107554.046875 ==
23
Stopping at epoch 1518 with Validation MSE: 4.16917610168457
== Test MSE: 4.419544696807861 ==
== Original Test MSE: 107554.046875 ==
naive mse: 21.15
retry
lr:  0.098 wd:  1e-07 h:  10
ANN
22
Stopping at epoch 3308 with Validation MSE: 0.011627701111137867
== Tes

Stopping at epoch 2388 with Validation MSE: 0.027138201519846916
== Test MSE: 0.02733168751001358 ==
== Original Test MSE: 665.1439819335938 ==
22
Stopping at epoch 2808 with Validation MSE: 4.1548590660095215
== Test MSE: 4.504594326019287 ==
== Original Test MSE: 109623.8125 ==
VIX
23
Stopping at epoch 2838 with Validation MSE: 3.8766093254089355
== Test MSE: 4.222072124481201 ==
== Original Test MSE: 102748.3359375 ==
23
Stopping at epoch 3918 with Validation MSE: 4.119603633880615
== Test MSE: 4.469552516937256 ==
== Original Test MSE: 108771.0234375 ==
Bloomberg
23
Stopping at epoch 2548 with Validation MSE: 4.119603633880615
== Test MSE: 4.469552516937256 ==
== Original Test MSE: 108771.0234375 ==
23
Stopping at epoch 3228 with Validation MSE: 0.02592380717396736
== Test MSE: 0.02473421022295952 ==
== Original Test MSE: 601.9317626953125 ==
naive mse: 21.15
retry
lr:  0.099 wd:  0.0001 h:  22
ANN
22
Stopping at epoch 3018 with Validation MSE: 0.04632658511400223
== Test MSE: 0.0

Stopping at epoch 3808 with Validation MSE: 0.0393078438937664
== Test MSE: 0.05592672526836395 ==
== Original Test MSE: 1396.15478515625 ==
22
Stopping at epoch 7898 with Validation MSE: 0.0388234406709671
== Test MSE: 0.06432639062404633 ==
== Original Test MSE: 1605.843994140625 ==
VIX
23
Stopping at epoch 5578 with Validation MSE: 3.9394145011901855
== Test MSE: 4.464469909667969 ==
== Original Test MSE: 111451.03125 ==
23
Stopping at epoch 5738 with Validation MSE: 3.9394145011901855
== Test MSE: 4.464469909667969 ==
== Original Test MSE: 111451.03125 ==
Bloomberg
23
Stopping at epoch 4438 with Validation MSE: 3.9394145011901855
== Test MSE: 4.464469909667969 ==
== Original Test MSE: 111451.03125 ==
23
Stopping at epoch 4848 with Validation MSE: 3.9394145011901855
== Test MSE: 4.464469909667969 ==
== Original Test MSE: 111451.03125 ==
naive mse: 21.15
retry
lr:  0.099 wd:  1e-07 h:  1
ANN
22
Stopping at epoch 15158 with Validation MSE: 2.106257438659668
== Test MSE: 2.30676770210

In [ ]:
tables = []
#d_name = ['ANN', 'VIX', 'Bloomberg']


num_epochs=1000000

lr = 0.001
wd = 1e-7
index = [no_sent, vix, bloom]


naive_mse = [21.15, 168.04, 510.74, 1903.75]
best_mse = [21.15, 168.04, 510.74, 1903.75]
best_no = [21.15, 168.04, 510.74, 1903.75]
best_vix = [21.15, 168.04, 510.74, 1903.75]
best_bloom = [21.15, 168.04, 510.74, 1903.75]

while best_mse[0] >=  11.289:
    for h in[1,5,10,22]:
        for ind in index:
            f'temp_m_{h}_{ind}', f'temp_mse_{h}_{ind}' = iterate(ind, num_epochs, h, lr, wd)
            f'temp_m_sch_{h}_{ind}', f'temp_mse_sch_{h}_{ind}' = iterate(ind, num_epochs, h, lr, wd, scheduler=True)
            print(f'naive mse: {naive_mse[0]}')


            if temp_mse_no < naive_mse[0]:
                torch.save(temp_m_no.state_dict(), f'./models/sy/1107_h1_ANN_wd{wd}_lr{lr}')
                tables.append([temp_mse_no, lr,wd])

            if temp_mse_sch < naive_mse[0]:
                torch.save(temp_m_sch.state_dict(), f'./models/sy/1107_h1_ANN_wd{wd}_lr{lr}_sch')
                tables.append([temp_mse_sch, lr,wd])  

            if temp_mse_no < best_mse[0]:
                best_mse[0] = temp_mse_no
                best_lr = lr
                best_wd = wd
                print('Best Model: ',temp_mse_no, lr, wd)
                best_model = temp_m_no

            if temp_mse_sch < best_mse_sch[0]:
                best_mse_sch[0] = temp_mse_sch
                best_lr_sch = lr
                best_wd_sch = wd
                print('Best Model: ',temp_mse_sch, lr, wd)
                best_model_sch = temp_m_sch

            else:
                print('retry')

    print(best_mse[0], best_lr, best_wd)
    print(best_mse_sch[0], best_lr_sch, best_wd_sch)

    torch.save(best_model.state_dict(), f'./models/sy/best/1107_h1_ANN_wd{best_wd}_lr{best_lr}')
    torch.save(best_model_sch.state_dict(), f'./models/sy/best/1107_h1_ANN_wd{best_wd_sch}_lr{best_lr_sch}_sch')


In [25]:
temp_all

[]